# Imports

In [1]:
import pandas as pd

In [2]:
import nltk
import re

from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mkavalio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mkavalio\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mkavalio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mkavalio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
import numpy as np
# def dummy_npwarn_decorator_factory():
#   def npwarn_decorator(x):
#     return x
#   return npwarn_decorator
# np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [4]:
from bertopic import BERTopic

c:\Users\mkavalio\code\gitdir\TopicModelling\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset import and text preprocessing

In [5]:
df = pd.read_csv(r"C:\Users\mkavalio\code\gitdir\TopicModelling\Data\Car_Reviews_Database_KaggleSentiment.csv", encoding='ISO-8859-1')
df = df.dropna()
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 234 entries, 0 to 277
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    234 non-null    int64 
 1   Model   234 non-null    object
 2   Review  234 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


In [6]:
stop_words = set(stopwords.words('english'))
add_to_stop = ["car", "buick", "cadillac", "chevrolet", "chrysler", "dodge", "ford", "gmc", "honda", "hyundai", "jeep", "kia", "mazda", \
               "nissan", "subaru", "toyota", "volkswagen", "volvo"]
for word in add_to_stop:
    stop_words.add(word)
print(stop_words)

def preprocess_text(text: str, stopwords=stop_words) -> str:
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text) # leave only abc characters and spaces
    
    # remove stopwords
  
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if w.lower() not in stopwords]
    text = " ".join(tokens)
    text = text.lower().strip() # removes leading and trailing whitespaces
    
    return text

{'toyota', "weren't", "shouldn't", 'honda', 'hyundai', 'o', "shan't", 'of', 'but', 'only', "you've", "wouldn't", 'shouldn', 'here', 'above', 'into', 'it', 'were', 'why', 'your', 'was', 'them', 'have', 'yours', 'whom', 'his', 'her', 'both', 'than', 'll', "isn't", 'few', 'doing', 'with', 'on', 'in', 'couldn', 'hers', 'further', 'car', 'or', 'wasn', 'other', "it's", 'these', "didn't", 'our', 'some', 'is', 'gmc', 're', 'hadn', "doesn't", 'as', 'down', 'who', 'off', 'don', 'below', 'i', 'such', 'being', 'all', 'too', 'now', 'that', 'between', 'no', 'theirs', 'so', 'mazda', 'myself', 'same', 'you', 'this', 'to', 'my', 'm', 've', "should've", 's', 'and', 'dodge', 'ours', 'over', 'under', 'very', "she's", 'himself', 'what', 'if', 'doesn', 'until', 'where', 't', 'just', 'yourself', 'be', 'am', "mightn't", 'd', 'he', 'before', 'hasn', 'again', 'are', 'when', 'then', 'itself', 'because', 'y', 'jeep', 'ourselves', 'didn', "aren't", "won't", 'has', 'each', 'mightn', 'subaru', 'we', 'at', 'she', 'wi

In [7]:
df['Review_processed'] = df['Review'].apply(lambda text: preprocess_text(text))
docs = df['Review_processed'].tolist()
df.sample(2)

,Year,Model,Review,Review_processed
22,2009,Hyundai,2009 Hyundai Sonata: Very poor gas mileage - b...,sonata poor gas mileage sticker minimum servic...
100,2014,Bentley,"Recently, I have seen a super luxurious car mo...",recently seen super luxurious model coming bri...


# Experiments

## BERTopic

### Basics (quick start)

In [8]:
from sentence_transformers import SentenceTransformer

In [9]:
topic_model = BERTopic()


In [10]:
topics, probs = topic_model.fit_transform(docs)

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/53aa51172d142c89d9012cce15ae4d6cc0ca6895895114379cacb4fab128d9db?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1739378592&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTM3ODU5Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zZW50ZW5jZS10cmFuc2Zvcm1lcnMvYWxsLU1pbmlMTS1MNi12Mi81M2FhNTExNzJkMTQyYzg5ZDkwMTJjY2UxNWFlNGQ2Y2MwY2E2ODk1ODk1MTE0Mzc5Y2FjYjRmYWIxMjhkOWRiP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=jUq~xACAlHVeF8EaCjPOhCc1mi-qCQSgPersUaPmxuy-q1PSdohfmHeglAtiQGOrv9Kf1VWYcVHd2y0G~ylQmX3Ztn5WaeVeXR1hOBLrI8y-YoVXunSjG6RfwIgaSDrTMGClTY2ryKXLQIiobtOIVvHDIeNsegYXkX-DGPzzYRr8e8J9QBUGUkKmKh6hMyNQUH5e09nczD9eUNUNoz9Wa6PV96DXV3vONGw3fSpFVc7bjrEMN-NwwbvQr2nXje1phGymFp496BdD~7WcRY-jUKmmN4xXbvR5z00EejmvxBq

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,44,-1_model_also_luxurious_world,"[model, also, luxurious, world, models, mulsan...",[recently seen super luxurious model coming br...
1,0,149,0_drive_like_driving_great,"[drive, like, driving, great, good, even, spee...",[vehicle long stood excess opulence knew going...
2,1,22,1_handling_engine_interior_camaro,"[handling, engine, interior, camaro, ranks, cl...",[despite athletic handling upscale cabin regal...
3,2,19,2_bmw_series_handling_luxury,"[bmw, series, handling, luxury, rear, small, s...",[bmw series muscular engines nimble handling a...


In [ ]:
topic_model.get_topic(0)

[('drive', np.float64(0.028464994770944746)),
 ('like', np.float64(0.02826070948020483)),
 ('driving', np.float64(0.02776705583973617)),
 ('great', np.float64(0.02667612446943073)),
 ('good', np.float64(0.02641275855629422)),
 ('even', np.float64(0.023116349820506067)),
 ('speed', np.float64(0.021511383988461412)),
 ('vehicle', np.float64(0.02138206730751355)),
 ('one', np.float64(0.02062697612355299)),
 ('well', np.float64(0.02038947237905064))]

In [ ]:
topic_model.get_document_info(docs)

### Fine-tune Topic Representations (basics)

In [ ]:
from bertopic.representation import KeyBERTInspired

In [ ]:
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)

In [ ]:
topics, probs = topic_model.fit_transform(docs)

OSError: sentence-transformers/all-MiniLM-L6-v2 does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,56,-1_luxurious_luxury_richest_bentley,"[luxurious, luxury, richest, bentley, cars, mo...",[recently seen super luxurious model coming br...
1,0,131,0_audi_vehicle_cars_driving,"[audi, vehicle, cars, driving, suv, highway, e...",[vehicle long stood excess opulence knew going...
2,1,27,1_automotive_regal_seats_luxurious,"[automotive, regal, seats, luxurious, sedan, c...",[despite athletic handling upscale cabin regal...
3,2,20,2_bmw_mercedes_sedan_luxury,"[bmw, mercedes, sedan, luxury, audi, luxurious...",[bmw series muscular engines nimble handling a...


### Inspired by https://medium.com/@pauloyc/topic-modelling-and-multiclass-text-classification-using-transformers-6fc3ee69c15d

In [11]:
from bertopic.representation import MaximalMarginalRelevance, KeyBERTInspired
from umap import UMAP

# All representation models
mmr_model = MaximalMarginalRelevance(diversity=0.3)
keybert_model = KeyBERTInspired()
umap_model = UMAP(random_state=42, n_jobs=-1)


In [12]:
# Create your representation model
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model,
}

In [13]:
from transformers import AutoModel, AutoTokenizer, pipeline

# Create your embedding model
model_path = r"C:\Users\mkavalio\code\gitdir\TopicModelling\Models\distilbertdistilbert-base-uncased"
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model from the local directory
embedding_model = pipeline("feature-extraction", model=model, tokenizer=tokenizer)

Device set to use cpu


In [14]:

# Use the representation model in BERTopic on top of the default pipeline
topic_model = BERTopic(
    embedding_model=embedding_model,
    language="english", 
    calculate_probabilities=True, 
    verbose=True, 
    representation_model=representation_model,
    umap_model=umap_model
    )

#topic_model.fit(docs)
topics, probs = topic_model.fit_transform(docs)

2025-02-12 16:48:51,766 - BERTopic - Embedding - Transforming documents to embeddings.
100%|██████████| 234/234 [00:10<00:00, 22.50it/s]
2025-02-12 16:49:02,187 - BERTopic - Embedding - Completed ✓
2025-02-12 16:49:02,187 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-12 16:49:11,010 - BERTopic - Dimensionality - Completed ✓
2025-02-12 16:49:11,011 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-12 16:49:11,029 - BERTopic - Cluster - Completed ✓
2025-02-12 16:49:11,032 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-12 16:49:21,467 - BERTopic - Representation - Completed ✓


In [15]:
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'ride | comfortable | good | even | seats', 64],
                                   [1, 'work | like | would | even | one', 30],
                                   [2, 'camaro | handling | class | interior | ranks', 22],
                                   [3, 'mpg | liter | highway | speed | average', 18],
                                   [4, 'handling | regal | engine | midsize | interior', 10]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'QB4WEgo=', 'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.04,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': 'oieqQP12tECvlFZA3xFUQITbgUA=', 'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': 'P6SNQegQi0Gt0NNBFp/ZQfly0kE=', 'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(2.8165522),
                                'y': np.float32(23.029423),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(4.651003),
                                'xshift': 10,
                                'y': np.float32(31.283081)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(4.651003),
                           'x1': np.float32(4.651003),
                           'y0': np.float32(14.775766),
                           'y1': np.float32(31.283081)},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(2.8165522),
                           'x1': np.float32(6.4854536),
                           'y0': np.float32(23.029423),
                           'y1': np.float32(23.029423)}],
               'sliders': [{'active': 0,
                            'pad': {'t': 50},
                            'steps': [{'args': [{'marker.color': [['red', '#B0BEC5', '#B0BEC5', '#B0BEC5', '#B0BEC5']]}],
                                       'label': 'Topic 0',
                                       'method': 'update'},
                                      {'args': [{'marker.color': [['#B0BEC5', 'red', '#B0BEC5', '#B0BEC5', '#B0BEC5']]}],
                                       'label': 'Topic 1',
                                       'method': 'update'},
                                      {'args': [{'marker.color': [['#B0BEC5', '#B0BEC5', 'red', '#B0BEC5', '#B0BEC5']]}],
                                       'label': 'Topic 2',
                                       'method': 'update'},
                                      {'args': [{'marker.color': [['#B0BEC5', '#B0BEC5', '#B0BEC5', 'red', '#B0BEC5']]}],
                                       'label': 'Topic 3',
                                       'method': 'update'},
                                      {'args': [{'marker.color': [['#B0BEC5', '#B0BEC5', '#B0BEC5', '#B0BEC5', 'red']]}],
                                       'label': 'Topic 4',
                           

In [16]:
topic_model.visualize_barchart()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.02132885922431856, 0.02140035879259243, 0.02159178992145769,
                    0.021991089499863437, 0.023069579035326002],
              'xaxis': 'x',
              'y': [seats  , even  , good  , comfortable  , ride  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03742759484520405, 0.039879923730376826,
                    0.04164867463631644, 0.04600087262496566, 0.05231865129553694],
              'xaxis': 'x2',
              'y': [one  , even  , would  , like  , work  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.08386357788981018, 0.0869466738788787, 0.09518137664545477,
                    0.12012459266648297, 0.13178562239827313],
              'xaxis': 'x3',
              'y': [ranks  , interior  , class  , handling  , camaro  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.041861746462933835, 0.04544175479196655,
                    0.05157952669980628, 0.06137720204602857, 0.08444267586858518],
              'xaxis': 'x4',
              'y': [average  , speed  , highway  , liter  , mpg  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.091522814609346, 0.09281069279087392, 0.09859533017777092,
                    0.10061889934702939, 0.13909163361382237],
              'xaxis': 'x5',
              'y': [interior  , midsize  , engine  , regal  , handling  ],
              'yaxis': 'y5'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 2',
                                'x': 0.6375000000000001,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 3',
                                'x': 0.9125,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 4',
                                'x': 0.0875,
                                'xanchor': 'center',

In [17]:
topic_model.get_topics()

{-1: [('model', np.float64(0.03102960363021273)),
  ('luxurious', np.float64(0.026785572761421687)),
  ('also', np.float64(0.025345076060515427)),
  ('driving', np.float64(0.024928899218665975)),
  ('drive', np.float64(0.024553542614465036)),
  ('like', np.float64(0.024358557313791336)),
  ('seats', np.float64(0.022366076051903497)),
  ('world', np.float64(0.022048040569710985)),
  ('bmw', np.float64(0.02165214445594563)),
  ('engine', np.float64(0.021131229206189418))],
 0: [('ride', np.float64(0.023069579035326002)),
  ('comfortable', np.float64(0.021991089499863437)),
  ('good', np.float64(0.02159178992145769)),
  ('even', np.float64(0.02140035879259243)),
  ('seats', np.float64(0.02132885922431856)),
  ('drive', np.float64(0.020786681845264134)),
  ('steering', np.float64(0.019611532017189653)),
  ('like', np.float64(0.01848834004299665)),
  ('great', np.float64(0.018457052512562234)),
  ('miles', np.float64(0.017955226215635098))],
 1: [('work', np.float64(0.05231865129553694)),
 

In [18]:
topic_model.get_topic_freq()

,Topic,Count
0,-1,90
1,0,64
4,1,30
2,2,22
3,3,18
5,4,10


In [19]:
repr_docs = topic_model.get_representative_docs(docs)

In [20]:
repr_docs

{-1: ['recently seen super luxurious model coming british auto mobile company bentley motors bentley mulsanne awesome world also readied expected price rs rs crore really superb model richest person likely able purchase richest model world many luxurious coming also high range model country different model luxurious models many luxurious maker like bmw audi mercedes benz also going makes different type cars world also like high cost models buy plan purchase marvellous model',
  'recently seen super luxurious model coming british auto mobile company bentley motors bentley mulsanne awesome world also readied expected price rs rs crore really superb model richest person likely able purchase richest model world many luxurious coming also high range model country different model luxurious models many luxurious maker like bmw audi mercedes benz also going makes different type cars world also like high cost models buy plan purchase marvellous model',
  'recently seen super luxurious model com

In [21]:
df.sample(1)

,Year,Model,Review,Review_processed
35,2016,Hyundai,Hyundai XG350: Very reliable up to ~130k miles...,xg reliable k miles started fall apart first g...


In [22]:
repr_docs[0][0]

'since delivered directly ordered factory elegance wrapped excellent sporty exterior sport package suspension little stiffer overly uncomfortable even roughest roads handled everything german autobahn interstates port maryland across united states engine delivers power although turbo seem pick around rpm comfortable ride whether across town heading rockies hours away issues developed small vacuum leak exhaust easily found fixed item voice navigation translation pick well dial addresses tedious gas mileage matches rated mileage shown original sticker although mine driven city type driving highway open winding roads dream drive sticks pavement ranges available take german roads mph speed limiter limit road like driving mph ride firm smooth would buy another one another audi next even second'

In [24]:
df[df['Review_processed']==repr_docs[0][0]]['Review']

85    I have had this car since delivered to me dire...
Name: Review, dtype: object

In [25]:
unproc_repr_docs = []
topic_number = 4

for doc in repr_docs[topic_number]:
    
    unpr_doc = df[df['Review_processed']==doc]['Review']
    unproc_repr_docs.append(unpr_doc)

In [26]:
unproc_repr_docs

[116    The 2015 Buick Regal finishes in the middle of...
 Name: Review, dtype: object,
 140    The 2017 Cadillac CTS has precise steering and...
 Name: Review, dtype: object,
 120    Despite its athletic handling and upscale cabi...
 Name: Review, dtype: object]

In [27]:
df.iloc[116]

Year                                                             2015
Model                                                           Buick
Review              The 2015 Buick Regal finishes in the middle of...
Review_processed    regal finishes middle midsize class upscale co...
Name: 116, dtype: object

In [28]:
topic_model.find_topics("light")

([2, 3, -1, 1, 4],
 [np.float64(0.326077148713769),
  np.float64(0.3126974646226249),
  np.float64(0.29656718831599094),
  np.float64(0.27355013187989113),
  np.float64(0.25193586175865323)])

In [29]:
repr = topic_model.get_topic_info()

In [30]:
DOCUMENTS = repr['Representative_Docs'][1:].values
KEYWORDS = repr['Representation'][1:].values


In [31]:
DOCUMENTS

array([list(['since delivered directly ordered factory elegance wrapped excellent sporty exterior sport package suspension little stiffer overly uncomfortable even roughest roads handled everything german autobahn interstates port maryland across united states engine delivers power although turbo seem pick around rpm comfortable ride whether across town heading rockies hours away issues developed small vacuum leak exhaust easily found fixed item voice navigation translation pick well dial addresses tedious gas mileage matches rated mileage shown original sticker although mine driven city type driving highway open winding roads dream drive sticks pavement ranges available take german roads mph speed limiter limit road like driving mph ride firm smooth would buy another one another audi next even second', 'bought absolutely love k months attest drive lot highway miles highway speeds thing quite gives false impression going fast several times first months looked saw pushing mph quickly sl

In [32]:
KEYWORDS

array([list(['ride', 'comfortable', 'good', 'even', 'seats', 'drive', 'steering', 'like', 'great', 'miles']),
       list(['work', 'like', 'would', 'even', 'one', 'new', 'good', 'get', 'vehicle', 'never']),
       list(['camaro', 'handling', 'class', 'interior', 'ranks', 'top', 'series', 'reliability', 'rating', 'features']),
       list(['mpg', 'liter', 'highway', 'speed', 'average', 'drive', 'yukon', 'better', 'eight', 'track']),
       list(['handling', 'regal', 'engine', 'midsize', 'interior', 'base', 'upscale', 'materials', 'rear', 'cabin'])],
      dtype=object)

### GPT locally to automate the process of generating names for each topic

In [33]:
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""

In [38]:
from ollama_api import OllamaClient

client = OllamaClient()
response = client.generate_completion(model="deepseek-r1:7b", prompt=prompt)
print(response)

ModuleNotFoundError: No module named 'ollama_api'